<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Final-Project-Check-in" data-toc-modified-id="Final-Project-Check-in-1">Final Project Check-in</a></span></li><li><span><a href="#Group-Name" data-toc-modified-id="Group-Name-2">Group Name</a></span></li><li><span><a href="#Student-Names" data-toc-modified-id="Student-Names-3">Student Names</a></span></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-4">Load Data</a></span></li><li><span><a href="#Fit-scikit-learn-model" data-toc-modified-id="Fit-scikit-learn-model-5">Fit scikit-learn model</a></span></li><li><span><a href="#Evaluation-Metric" data-toc-modified-id="Evaluation-Metric-6">Evaluation Metric</a></span></li></ul></div>

Final Project Check-in
------

# Evan Chen
----------





Research Question / Hypothesis
----

## 1) Find the features that best predicts SalePrice of a house


Load Data
-----

In [197]:
from   category_encoders          import *
import numpy as np
import pandas as pd
from   sklearn.compose            import *
from   sklearn.ensemble           import RandomForestRegressor, StackingRegressor, BaggingRegressor
from   sklearn.experimental       import enable_iterative_imputer
from   sklearn.impute             import *
from   sklearn.linear_model       import *
from   sklearn.metrics            import mean_squared_log_error, mean_squared_error, mean_absolute_error # We have not covered it yet in class. The basics - AUC is from 0 to 1 and higher is better.
from   sklearn.pipeline           import Pipeline, FeatureUnion
from   sklearn.preprocessing      import *
from   sklearn.model_selection    import train_test_split
from sklearn.model_selection      import cross_val_score
from sklearn.inspection            import permutation_importance
from sklearn.base               import BaseEstimator, TransformerMixin
from sklearn.compose            import make_column_transformer
from sklearn.ensemble            import BaggingRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.model_selection    import RandomizedSearchCV
from pandas_profiling import ProfileReport


In [198]:
# Load Data
df = pd.read_csv('train.csv')
y = df.SalePrice.values
X = df.loc[:, df.columns!= 'SalePrice']

# Change 'object' types to numeric
X[['BsmtFullBath', 'FullBath', 'HalfBath', 'Fireplaces']] = X[['BsmtFullBath', 'FullBath', 'HalfBath', 'Fireplaces']].apply(pd.to_numeric)


# Categorical Ordinal
sale_cond = ['SaleCondition']
sale_type = ['SaleType']
qual_cat_ord = ['ExterQual', 'KitchenQual']
functional_col = ['Functional']

# Date Columns
year_cols = ['YrSold', 'YearBuilt', 'YearRemodAdd']

# OHE columns
OHE_cols = ['CentralAir', 'Neighborhood']

num_features =['LotArea', 'OverallQual', 'OverallCond',
       'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea',
       'BsmtFullBath', 'FullBath', 'HalfBath', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageArea', 'WoodDeckSF', 'PoolArea', 'MiscVal', 'LotFrontage']

# Change Dataframe to only contain features that we want
num_features = num_features + functional_col + qual_cat_ord + year_cols + sale_cond + sale_type
feature_cols = num_features + OHE_cols
X = X[feature_cols]

# Split to train/test
X, X_test, y, y_test = train_test_split(X,y, test_size = 0.2)


/Users/Evan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


# DateTime Encoding: YrSold, YearBuilt, YearRemod,  = categorical ordinal. The newest is better

In [199]:
def map_dates(X):
    return [2020- year for year in X.values] 

transformer_age = FunctionTransformer(map_dates)

# Custom Transform for Categorical

In [200]:
# Functions to turn into FunctionTransformer
def map_SaleCond(X):
    "Replace value in array with the mapped value"
    map_dict = {'Normal' : 2,
                'Abnorml': 0,
                'Partial': 1,
                'Alloca' : 1,
                'AdjLand': 1,
                'Family': 1}
    return [map_dict[value] for value in X]

def map_SaleType(X):
    "Replace value in array with the mapped value"
    map_dict = {'New' : 1,
                'WD': 0,
                'COD': 0,
                'Con' : 0,
                'ConLD': 0,
                'ConLw': 0,
                'CWD' : 0,
                'ConLI': 0,
               'Oth' : 0}

    return [map_dict[value] for value in X]

# Change functions to transformers
transformer_SaleCond = FunctionTransformer(map_SaleCond)
transformer_SaleType = FunctionTransformer(map_SaleType)


# Transforming Ordinal Categorical and Preprocessing

In [201]:
# Fa=Fair, Gd=Good, TA=Typical, Ex=Excellent ranked in order
quality_enc = OrdinalEncoder(categories=[['Fa', 'Gd', 'TA', 'Ex']])
X['ExterQual'] = quality_enc.fit_transform(X[['ExterQual']])
X['KitchenQual'] =  quality_enc.fit_transform(X[['KitchenQual']])

# How well the house functions
functional_enc = OrdinalEncoder(categories=[['Sev', 'Maj2', 'Mod', 'Min2', 'Maj1', 'Min1', 'Typ']])
X['Functional'] = functional_enc.fit_transform(X[['Functional']])

# Transform year columns to subtact from day
for col in year_cols:
    X[col] = transformer_age.fit_transform(X[col])

# Transform SaleType and SaleCondition columns
X['SaleType'] = transformer_SaleType.fit_transform(X['SaleType'].values)
X['SaleCondition'] = transformer_SaleCond.fit_transform(X['SaleCondition'].values)

# Numerical preprocessing
num_pipe = Pipeline([('imputer', SimpleImputer(strategy='median')), #  add_indicator=True)),
                    ('scaler', StandardScaler()),
                    ('QT', QuantileTransformer(n_quantiles=100, output_distribution='normal'))
                    ])


# Fit Initial Model

In [202]:
# MAPE Function
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# List of Regressors to try
regressors = [LinearRegression(), Lasso(), LassoCV(), Ridge(), RidgeCV(),
              ElasticNet(), BayesianRidge(), 
            GradientBoostingRegressor(), RandomForestRegressor(), SGDRegressor(), AdaBoostRegressor()]

# Search regressors for lowest loss metric
for reg in regressors: 
    
    # Lists to calculate the average MAE and RMSE
    MAE_avg = []
    RMSE_avg = []
    MAPE_avg = []
    
    # Preprocessor
    prep = ColumnTransformer(
        transformers=[
            ('OHE', OneHotEncoder(handle_unknown='ignore'), OHE_cols),
            ('num', num_pipe, num_features) ])
    # Pipeline
    pipe = Pipeline([('prep', prep),
                     ('model', reg)])

    # Iterate to get RMSEs and MAEs across different splits
    for i in range(10): 
        X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = 0.2)
        y_train_log = np.log(y_train)
        y_val_log = np.log(y_val)
        
        # Fit Model for MAE
        model_1 = pipe.fit(X_train, y_train)
        y_pred_1 = pipe.predict(X_val)
        
        # Fit Model for Log_RMSE
        model_2 = pipe.fit(X_train, y_train_log)
        y_pred_2 = pipe.predict(X_val)
        
        # Get Losses
        mae = mean_absolute_error(y_val, y_pred_1)
        mape = mean_absolute_percentage_error(y_val, y_pred_1)
        rmse_log = np.sqrt(mean_squared_error(y_val_log, y_pred_2))
        # Add losses to array
        MAPE_avg.append(mape)
        MAE_avg.append(mae)
        RMSE_avg.append(rmse_log)
    print("MODEL", reg, "AVG MAE:", np.mean(MAE_avg), "RMSE_log:", np.mean(RMSE_avg), "AVG MAPE:", np.mean(MAPE_avg))


MODEL LinearRegression() AVG MAE: 21271.913428021842 RMSE_log: 0.13797731436524036 AVG MAPE: 13.04065380477186


/Users/Evan/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 50056357280.21478, tolerance: 498271033.57073164
  model = cd_fast.enet_coordinate_descent(
/Users/Evan/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 11176373051.332031, tolerance: 575653712.2288579
  model = cd_fast.enet_coordinate_descent(
/Users/Evan/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 17470656284.795776, tolerance: 550014979.6167637
  model = cd_fast.enet_coordinate_descent(
/Users/Evan/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_mode

MODEL Lasso() AVG MAE: 21853.738441568225 RMSE_log: 0.40777248235476576 AVG MAPE: 13.062634493497143
MODEL LassoCV() AVG MAE: 22486.246652020833 RMSE_log: 0.1489549123889992 AVG MAPE: 13.302407974224526
MODEL Ridge() AVG MAE: 20885.737116582215 RMSE_log: 0.12878704057362528 AVG MAPE: 12.158541760427742
MODEL RidgeCV(alphas=array([ 0.1,  1. , 10. ])) AVG MAE: 21236.508135266027 RMSE_log: 0.13882966287597703 AVG MAPE: 13.044914244722738
MODEL ElasticNet() AVG MAE: 24334.46037267516 RMSE_log: 0.38761613108331694 AVG MAPE: 14.870454275819549
MODEL BayesianRidge() AVG MAE: 21652.497124101767 RMSE_log: 0.14592352679439266 AVG MAPE: 13.461921178699715
MODEL GradientBoostingRegressor() AVG MAE: 17323.773054938414 RMSE_log: 0.1415705673849744 AVG MAPE: 9.968161215528053
MODEL RandomForestRegressor() AVG MAE: 19387.949173789173 RMSE_log: 0.146613380275776 AVG MAPE: 11.090123879752879
MODEL SGDRegressor() AVG MAE: 22487.020848440097 RMSE_log: 0.36253878729073985 AVG MAPE: 14.24770923564979
MODEL 

# Gradient Boosting Regressor and RandomForest performs the best

In [203]:
# Helper class
class DummyEstimator(BaseEstimator):
    "Pass through class, methods are present but do nothing."
    def fit(self): pass
    def score(self): pass
    
# Placeholder for Searching Hyperparameters
dummy_pipe = Pipeline([
                      ('prep', prep),
                    ('reg', DummyEstimator())]) # Placeholder Estimator

# Hyperparameter Turning
#### learning rate: must iteratively guess

In [204]:
# Random Forest Hyperparameter tuning , set random state for consistency
tree_hyp = dict(reg        = [RandomForestRegressor(random_state=0)],
                reg__max_depth        = range(1, 10),
                reg__max_features = ['log2', 'auto', 'sqrt'],
               reg__min_samples_leaf = range(3, 10),
               reg__min_samples_split = range(3, 10),
                reg__n_estimators = [100,200,300])

# Gradient Boosting Hyperparameter tuning,  set random state for consistency
boost_hyp = dict(reg        = [GradientBoostingRegressor(random_state=0)],
                 reg__learning_rate        = [0.0001, 0.001, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2],
               reg__criterion        = ['mse', 'friedman_mse'],
                reg__max_features = ['log2', 'auto', 'sqrt'],
               reg__min_samples_leaf = range(3, 10),
               reg__min_samples_split = range(3, 10),
                 reg__validation_fraction =  [0.1, 0.15, 0.2],
                reg__n_estimators = [100,200,300])

# Merge Search Space
search_space = [tree_hyp, boost_hyp]

# Ranodmized CV search
clf_algos_rand = RandomizedSearchCV(estimator=dummy_pipe, 
                                    param_distributions=search_space, 
                                    n_iter=100,
                                    cv=5, 
                                    n_jobs=-1,
                                    verbose=1,
                                   scoring='r2')
# Get the best model 
result = clf_algos_rand.fit(X_train, y_train)
best_model = result.best_estimator_

#Get the best hyperparams
tuned_hyperparams = result.best_params_
print(tuned_hyperparams)
print(f"The validation set - {result.best_score_}")

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   31.6s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.4min finished


{'reg__validation_fraction': 0.2, 'reg__n_estimators': 200, 'reg__min_samples_split': 8, 'reg__min_samples_leaf': 5, 'reg__max_features': 'log2', 'reg__learning_rate': 0.1, 'reg__criterion': 'mse', 'reg': GradientBoostingRegressor(criterion='mse', max_features='log2',
                          min_samples_leaf=5, min_samples_split=8,
                          n_estimators=200, random_state=0,
                          validation_fraction=0.2)}
The validation set - 0.8780864321495413


# Final Model Fitting

In [205]:
tuned_hyperparams

{'reg__validation_fraction': 0.2,
 'reg__n_estimators': 200,
 'reg__min_samples_split': 8,
 'reg__min_samples_leaf': 5,
 'reg__max_features': 'log2',
 'reg__learning_rate': 0.1,
 'reg__criterion': 'mse',
 'reg': GradientBoostingRegressor(criterion='mse', max_features='log2',
                           min_samples_leaf=5, min_samples_split=8,
                           n_estimators=200, random_state=0,
                           validation_fraction=0.2)}

In [206]:
# Get default 
default = GradientBoostingRegressor().get_params()
tuned = default

# Assign tuned hyperparameter values
tuned['validation_fraction'] = tuned_hyperparams['reg__validation_fraction']
tuned['n_estimators'] = tuned_hyperparams['reg__n_estimators']
tuned['min_samples_leaf'] = tuned_hyperparams['reg__min_samples_leaf']
tuned['min_samples_split'] = tuned_hyperparams['reg__min_samples_split']
tuned['max_features'] = tuned_hyperparams['reg__max_features']
tuned['learning_rate'] = tuned_hyperparams['reg__learning_rate']
tuned['criterion'] = tuned_hyperparams['reg__criterion']


In [207]:
# Pipeline
pipe = Pipeline([('prep', prep),
                 ('model', GradientBoostingRegressor(**tuned))])
    
# Log Transform
y_train_log = np.log(y_train)
y_val_log = np.log(y_val)

# Fit Model for MAE
model_1 = pipe.fit(X_train, y_train)
y_pred_1 = pipe.predict(X_val)

# Fit Model for Log_RMSE
model_2 = pipe.fit(X_train, y_train_log)
y_pred_2 = pipe.predict(X_val)

# Get Losses
mape = mean_absolute_percentage_error(y_val, y_pred_1)
mae = mean_absolute_error(y_val, y_pred_1)
rmse_log = np.sqrt(mean_squared_error(y_val_log, y_pred_2))
print("TEST MAE:", mae, "Log RMSE:", rmse_log, "TEST MAPE:", mape)

TEST MAE: 18469.670914198967 Log RMSE: 0.1396350373650737 TEST MAPE: 9.62202567554539


# Discussion

### MAE: 16,759.92 means that our predictions were on average 16,759 dollars away from the true house price in the test set. The MAPE shows that this 16759 is equal to a 10.04% error from the true house price. Log RMSE: 0.125 is the Root-Mean-Squared-Error (RMSE) between the logarithm of the predicted value and the logarithm of the observed sales price. Taking logs means that errors in predicting expensive houses and cheap houses will affect the result equally.


# Lessons Learned
### Features Selection: There could have been more time spent on feature selection by running permutation importances on all features preprocessing models; however, I had some difficulty getting the Column Transformers to work in the Pipeline. Due to the amount of categorical features with high cardinality, I had to carefully select the ones that I wanted to encode. It turns out that the "Neighborhood" column was very important when I ran a permutation importance test on my feature columns; however, it had 24 unique variables and encoding more would cause the curse of dimmensionality

### Feature Engineering: In the future, more time could be spent on extracting more information from the unique variables for each categorical column. Perhaps PCA would be useful here.